📌 Extracción(E - Extract)

In [2]:
#extraemos la data en formato json desde el API. Verificamos su estructura basica y configuracion
import pandas as pd

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'

df = pd.read_json(url)

df.head()
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


(7267, 6)

🔧 Transformación (T - Transform)

In [3]:
#se necesita desanidar todas las columnas, por lo que cada columna es enviada a un diccionario y luego plasmada en una nueva tabla total
df_total = pd.json_normalize(df.to_dict(orient='records'))

df_total.head()
df_total.info()
df_total.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

(7267, 21)

In [4]:
#Hacemos una comprobación de incoherencias en los datos

#valores nulos: No se ubican valores nulos
df_total.isnull().sum().sort_values(ascending=False)

#registros duplicados, Primero verificamos por lineas y luego por CustomerID: No se ubican valores duplicados.
df_total.duplicated().sum()
df_total['customerID'].duplicated().sum()

#errores de formato por celdas: Se verifica que account.Charges.Total esta como object, sin embargo deberia ser una columna float
df_total.dtypes

#verificacion de columnas categoricas. 
    #- Churn tiene datos en blanco a parte de "no" y "yes"
    #- phone.MultipleLines tiene el dato "No phone service" a parte de "no" y "yes"
    #- internet.OnlineSecurity tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.OnlineBackup tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.DeviceProtection tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.TechSupport tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.StreamingTV tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.StreamingMovies tiene el dato "No internet service" a parte de "no" y "yes"

columnas_categoricas = df_total.select_dtypes(include='object').columns

for col in columnas_categoricas:
    print(f"\n{col}")
    print(df_total[col].value_counts())


customerID
customerID
0002-ORFBO    1
0003-MKNFE    1
0004-TLHLJ    1
0011-IGKFF    1
0013-EXCHZ    1
             ..
9987-LUTYD    1
9992-RRAMN    1
9992-UJOEL    1
9993-LHIEB    1
9995-HOTOH    1
Name: count, Length: 7267, dtype: int64

Churn
Churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

customer.gender
customer.gender
Male      3675
Female    3592
Name: count, dtype: int64

customer.Partner
customer.Partner
No     3749
Yes    3518
Name: count, dtype: int64

customer.Dependents
customer.Dependents
No     5086
Yes    2181
Name: count, dtype: int64

phone.PhoneService
phone.PhoneService
Yes    6560
No      707
Name: count, dtype: int64

phone.MultipleLines
phone.MultipleLines
No                  3495
Yes                 3065
No phone service     707
Name: count, dtype: int64

internet.InternetService
internet.InternetService
Fiber optic    3198
DSL            2488
No             1581
Name: count, dtype: int64

internet.OnlineSecurity
internet.OnlineSecurity
No  

In [5]:
#Manejo de inconsistencias

#convertimos account.Charges.Total en columna numerica, estandarizando los campos en blanco con NaN

df_total['account.Charges.Total'] = (
    pd.to_numeric(df_total['account.Charges.Total'], errors='coerce')
)

df_total['account.Charges.Total'].fillna(0, inplace=True)

#normalizacion de variables categoricas: 
    #- columna Churn

df_total['Churn'] = df_total['Churn'].replace(' ', 'No')

    #- columna phone.MultipleLines

df_total['phone.MultipleLines'] = (
    df_total['phone.MultipleLines'].replace('No phone service', 'No')
)

    #- columnas que contienen "No internet service" en lugar de "No"

internet_cols = [
    'internet.OnlineSecurity',
    'internet.OnlineBackup',
    'internet.DeviceProtection',
    'internet.TechSupport',
    'internet.StreamingTV',
    'internet.StreamingMovies'
]

df_total[internet_cols] = df_total[internet_cols].replace('No internet service', 'No'
)

#validameos correcciones

for col in columnas_categoricas:
    print(f"\n{col}")
    print(df_total[col].value_counts())

df_total.dtypes



customerID
customerID
0002-ORFBO    1
0003-MKNFE    1
0004-TLHLJ    1
0011-IGKFF    1
0013-EXCHZ    1
             ..
9987-LUTYD    1
9992-RRAMN    1
9992-UJOEL    1
9993-LHIEB    1
9995-HOTOH    1
Name: count, Length: 7267, dtype: int64

Churn
Churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

customer.gender
customer.gender
Male      3675
Female    3592
Name: count, dtype: int64

customer.Partner
customer.Partner
No     3749
Yes    3518
Name: count, dtype: int64

customer.Dependents
customer.Dependents
No     5086
Yes    2181
Name: count, dtype: int64

phone.PhoneService
phone.PhoneService
Yes    6560
No      707
Name: count, dtype: int64

phone.MultipleLines
phone.MultipleLines
No     4202
Yes    3065
Name: count, dtype: int64

internet.InternetService
internet.InternetService
Fiber optic    3198
DSL            2488
No             1581
Name: count, dtype: int64

internet.OnlineSecurity
internet.OnlineSecurity
No     5189
Yes    2078
Name: count, dtype: int64

inte

customerID                    object
Churn                         object
customer.gender               object
customer.SeniorCitizen         int64
customer.Partner              object
customer.Dependents           object
customer.tenure                int64
phone.PhoneService            object
phone.MultipleLines           object
internet.InternetService      object
internet.OnlineSecurity       object
internet.OnlineBackup         object
internet.DeviceProtection     object
internet.TechSupport          object
internet.StreamingTV          object
internet.StreamingMovies      object
account.Contract              object
account.PaperlessBilling      object
account.PaymentMethod         object
account.Charges.Monthly      float64
account.Charges.Total        float64
dtype: object

In [7]:
# Creación de la columna Cuentas_diarias
    # normalizamos el numero de días del mes a 30

df_total['Cuentas_Diarias'] = df_total['account.Charges.Monthly'] / 30

    #hacemos un redondeo a 2 decimales
    
df_total['Cuentas_Diarias'] = df_total['Cuentas_Diarias'].round(2)



In [ ]:
    # Hacemos las verificaciones de la ultima columna creada

df_total[['account.Charges.Monthly', 'Cuentas_Diarias']].head()

df_total[['account.Charges.Monthly', 'Cuentas_Diarias']].dtypes


account.Charges.Monthly    float64
Cuentas_Diarias            float64
dtype: object

In [17]:
# Estandarización y transformación de datos
    # Como priimer paso, vamos a transformar las columnas binarias por 1 y 0 en caso sea YES o NO respectivamente

columnas_binarias = [
    'Churn',
    'customer.Partner',
    'customer.Dependents',
    'phone.PhoneService',
    'phone.MultipleLines',
    'internet.OnlineSecurity',
    'internet.OnlineBackup',
    'internet.DeviceProtection',
    'internet.TechSupport',
    'internet.StreamingTV',
    'internet.StreamingMovies',
    'account.PaperlessBilling'
]

df_total[columnas_binarias] = df_total[columnas_binarias].replace({
    'Yes': 1,
    'No': 0
})


C:\Users\david\AppData\Local\Temp\ipykernel_6788\613684423.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_total[columnas_binarias] = df_total[columnas_binarias].replace({


In [18]:
    # ahora procedemos al renombramiento de columnas según el diccionario dado

renombre_columnas = {
    'customerID': 'id_cliente',
    'Churn': 'abandono_cliente',
    'customer.gender': 'genero',
    'customer.SeniorCitizen': 'adulto_mayor',
    'customer.Partner': 'tiene_pareja',
    'customer.Dependents': 'tiene_dependientes',
    'customer.tenure': 'antiguedad_meses',
    'phone.PhoneService': 'servicio_telefonico',
    'phone.MultipleLines': 'lineas_multiples',
    'internet.InternetService': 'servicio_internet',
    'internet.OnlineSecurity': 'seguridad_online',
    'internet.OnlineBackup': 'respaldo_online',
    'internet.DeviceProtection': 'proteccion_dispositivo',
    'internet.TechSupport': 'soporte_tecnico',
    'internet.StreamingTV': 'streaming_tv',
    'internet.StreamingMovies': 'streaming_peliculas',
    'account.Contract': 'tipo_contrato',
    'account.PaperlessBilling': 'facturacion_digital',
    'account.PaymentMethod': 'metodo_pago',
    'account.Charges.Monthly': 'cargo_mensual',
    'account.Charges.Total': 'cargo_total',
    'Cuentas_Diarias': 'cargo_diario'
}

df_total = df_total.rename(columns=renombre_columnas)


In [19]:
    # validamos el renombramiento

df_total.info()
df_total.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_cliente              7267 non-null   object 
 1   abandono_cliente        7267 non-null   object 
 2   genero                  7267 non-null   object 
 3   adulto_mayor            7267 non-null   int64  
 4   tiene_pareja            7267 non-null   int64  
 5   tiene_dependientes      7267 non-null   int64  
 6   antiguedad_meses        7267 non-null   int64  
 7   servicio_telefonico     7267 non-null   int64  
 8   lineas_multiples        7267 non-null   int64  
 9   servicio_internet       7267 non-null   object 
 10  seguridad_online        7267 non-null   int64  
 11  respaldo_online         7267 non-null   int64  
 12  proteccion_dispositivo  7267 non-null   int64  
 13  soporte_tecnico         7267 non-null   int64  
 14  streaming_tv            7267 non-null   

,id_cliente,abandono_cliente,genero,adulto_mayor,tiene_pareja,tiene_dependientes,antiguedad_meses,servicio_telefonico,lineas_multiples,servicio_internet,...,proteccion_dispositivo,soporte_tecnico,streaming_tv,streaming_peliculas,tipo_contrato,facturacion_digital,metodo_pago,cargo_mensual,cargo_total,cargo_diario
0,0002-ORFBO,0,Female,0,1,1,9,1,0,DSL,...,0,1,1,0,One year,1,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,0,Male,0,0,0,9,1,1,DSL,...,0,0,0,1,Month-to-month,0,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,1,Male,0,0,0,4,1,0,Fiber optic,...,1,0,0,0,Month-to-month,1,Electronic check,73.9,280.85,2.46
3,0011-IGKFF,1,Male,1,1,0,13,1,0,Fiber optic,...,1,0,1,1,Month-to-month,1,Electronic check,98.0,1237.85,3.27
4,0013-EXCHZ,1,Female,1,1,0,3,1,0,Fiber optic,...,0,1,1,0,Month-to-month,1,Mailed check,83.9,267.40,2.80
